# Analysis Ready Data Tutorial Part 2: Use Case 1

Time-series analysis (e.g. change detection and trend detection) is a powerful application of satellite imagery. However, a great deal of processing is required to prepare imagery for analysis. Analysis Ready Data (ARD), preprocessed time-series stacks of overhead imagery, allow for time-series analysis without any additional processing of the imagery. See [Analysis Data Defined](https://medium.com/planet-stories/analysis-ready-data-defined-5694f6f48815) for an excellent introduction and discussion on ARD.

In [Part 1](ard_1_intro_and_best_practices.ipynb) of this tutorial, we introduced ARD and covered the how and whys of using the Data and Orders APIs to create and interpret ARD.

This second part of the tutorial focuses on the first of two use cases. The use case addressed in this tutorial is:

* As a software engineer at an ag-tech company, I'd like to be able to order Planet imagery programmatically in a way that enables the data scientist at my organization to create time-series algorithms (e.g. monitoring ndvi curves over time) without further data cleaning and processing.

Please see the first part of the tutorial for an introduction to the Data and Orders APIs along with best practices. A lot of functionality developed in that tutorial will be copied here in a compact form.

## Introduction

Two things are interesting about this use case. First, we are calculating NDVI, and second, we are compositing scenes together. What is NDVI and what is compositing and why do we want to do it?

Great questions!

First, NDVI stands for normalized difference vegitation index. It is used a **LOT** to find out if vegetation is growing. You can find out more about NDVI at [USGS](https://www.usgs.gov/land-resources/eros/phenology/science/ndvi-foundation-remote-sensing-phenology?qt-science_center_objects=0#qt-science_center_objects) and [Wikipedia](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index). What we care about here is that NDVI uses the red and near-infrared bands of an image and returns one band with values that range from -1 to 1. So, we expect a single-band image for each order.

Compositing is a way to stitch multiple scenes together for maximum coverage. We want this because for a time series, we just want one image for each date and we want that one image to have the most coverage to minimize holes in our data. The composite tool takes in multiple scenes and returns one image. If we feed it scenes from a whole timestack, we still just get one image back! So, to avoid that disaster, we group our scenes by date and only composite the scenes that were collected on the same date.


## Implementation

The use case we will cover is: *As a software engineer at an ag-tech company, I'd like to be able to order Planet imagery programmatically in a way that enables the data scientist at my organization to create time-series algorithms (e.g. monitoring ndvi curves over time) without further data cleaning and processing.*

For this use case, the area of interest and time range are not specified. The need for no further processing indicates we should specify a strict usable pixel data filter. For time-series analysis the daily coverage of PS satellites is ideal. For our time-series analysis, we would like a single image that covers the entire area of interest (AOI). However, it may take multiple scenes to cover the entire AOI. Therefore, we will use the Composite tool to make a composite for each day in the time series analysis. This is a little tricky because the Composite tool just composites all of the scenes associated with the ids ordered. So we need to parse the scene ids we got from the Data API to get scene ids for each day, then submit an order for each day.

To summarize, these are the steps:
1. [Initialize API client](#Step-1:-Initialize-API-client)
1. [Search Data API](#Step-2:-Search-Data-API)
1. [Group IDs by Date](#Step-3:-Group-IDs-by-Date)
1. [Submit Orders](#Step-4:-Submit-Orders)
1. [Download Orders](#Step-5:-Download-Orders)
1. [Unzip and Verify Orders](#Step-6:-Unzip-and-Verify-Orders)

Note that, due to the processing-intensiveness of visualizing the NDVI images and UDM2s, we will be covering visualization in the next notebook, [Analysis Ready Data Tutorial Part 2: Use Case 1 - Visualization](ard_2_use_case_1_visualize_images.ipynb)

#### Import Dependencies

In [1]:
from copy import copy
import datetime
import os
from pathlib import Path
from pprint import pprint
import shutil
import time
from zipfile import ZipFile

import numpy as np
from planet import api
from planet.api import downloader, filters

#### Step 1: Initialize API client

In [2]:
# if your Planet API Key is not set as an environment variable, you can paste it below
API_KEY = os.environ.get('PL_API_KEY', 'your API Key goes here!!!!')

client = api.ClientV1(api_key=API_KEY)

#### Step 2: Search Data API

The goal of this step is to get the scene ids that meet the search criteria for this use case.

In [3]:
# these functions were developed in the best practices tutorial (part 1)
# create an api request from the search specifications
def build_request(aoi_geom, start_date, stop_date):
    '''build a data api search request for clear PSScene4Band imagery'''
    query = filters.and_filter(
        filters.geom_filter(aoi_geom),
        filters.range_filter('clear_percent', gte=90),
        filters.date_range('acquired', gt=start_date),
        filters.date_range('acquired', lt=stop_date)
    )
    return filters.build_search_request(query, ['PSScene4Band'])

def search_data_api(request, client, limit=500):
    result = client.quick_search(request)
    
    # this returns a generator
    return result.items_iter(limit=limit)

In [4]:
# define test data for the filter
test_start_date = datetime.datetime(year=2021,month=1,day=16)
test_stop_date = datetime.datetime(year=2021,month=4,day=1)

# iowa crops aoi
test_aoi_geom = {"type": "Polygon","coordinates": [[[152.88312435150146,-27.382590190112836],[152.87668704986572,-27.3816375134553],[152.86853313446045,-27.386553236946078],[152.87179470062256,-27.393107195074954],[152.873854637146,-27.3949361373168],[152.88200855255127,-27.3887633360014],[152.88312435150146,-27.382590190112836]]]}



In [5]:
request = build_request(test_aoi_geom, test_start_date, test_stop_date)
print(request)

{'item_types': ['PSScene4Band'], 'filter': {'type': 'AndFilter', 'config': ({'field_name': 'geometry', 'type': 'GeometryFilter', 'config': {'type': 'Polygon', 'coordinates': [[[152.88312435150146, -27.382590190112836], [152.87668704986572, -27.3816375134553], [152.86853313446045, -27.386553236946078], [152.87179470062256, -27.393107195074954], [152.873854637146, -27.3949361373168], [152.88200855255127, -27.3887633360014], [152.88312435150146, -27.382590190112836]]]}}, {'field_name': 'clear_percent', 'type': 'RangeFilter', 'config': {'gte': 90}}, {'field_name': 'acquired', 'type': 'DateRangeFilter', 'config': {'gt': '2021-01-16T00:00:00Z'}}, {'field_name': 'acquired', 'type': 'DateRangeFilter', 'config': {'lt': '2021-04-01T00:00:00Z'}})}}


In [6]:
items = list(search_data_api(request, client))
print(len(items))

12


#### Step 3: Group IDs by Date

In [7]:
# check out an item just for fun
pprint(items[0])

{'_links': {'_self': 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20210330_231606_45_2233',
            'assets': 'https://api.planet.com/data/v1/item-types/PSScene4Band/items/20210330_231606_45_2233/assets/',
            'thumbnail': 'https://tiles.planet.com/data/v1/item-types/PSScene4Band/items/20210330_231606_45_2233/thumb'},
 '_permissions': ['assets.analytic:download',
                  'assets.analytic_dn:download',
                  'assets.analytic_dn_xml:download',
                  'assets.analytic_sr:download',
                  'assets.analytic_xml:download',
                  'assets.basic_analytic:download',
                  'assets.basic_analytic_dn:download',
                  'assets.basic_analytic_dn_nitf:download',
                  'assets.basic_analytic_dn_rpc:download',
                  'assets.basic_analytic_dn_rpc_nitf:download',
                  'assets.basic_analytic_dn_xml:download',
                  'assets.basic_analytic_dn_xml_nitf:do

In [8]:
# item = items[0]
# acquired_date = item['properties']['acquired'].split('T')[0]
# acquired_date

In [9]:
def get_acquired_date(item):
    return item['properties']['acquired'].split('T')[0]

acquired_dates = [get_acquired_date(item) for item in items]

In [10]:
unique_acquired_dates = set(acquired_dates)
unique_acquired_dates

{'2021-02-20',
 '2021-02-22',
 '2021-03-04',
 '2021-03-07',
 '2021-03-13',
 '2021-03-24',
 '2021-03-25',
 '2021-03-27',
 '2021-03-30'}

In [11]:
def get_date_item_ids(date, all_items):
    return [i['id'] for i in all_items if get_acquired_date(i) == date]

def get_ids_by_date(items):
    acquired_dates = [get_acquired_date(item) for item in items]
    unique_acquired_dates = set(acquired_dates)
    
    ids_by_date = dict((d, get_date_item_ids(d, items))
                       for d in unique_acquired_dates)
    return ids_by_date
    
ids_by_date = get_ids_by_date(items)
pprint(ids_by_date)

{'2021-02-20': ['20210220_231641_0e20', '20210220_220400_1_0f46'],
 '2021-02-22': ['20210222_220810_0f24'],
 '2021-03-04': ['20210304_233410_1013',
                '20210304_231833_34_2231',
                '20210304_231830_99_2231'],
 '2021-03-07': ['20210307_220151_1052'],
 '2021-03-13': ['20210313_220118_104e'],
 '2021-03-24': ['20210324_231629_04_225a'],
 '2021-03-25': ['20210325_001549_25_1061'],
 '2021-03-27': ['20210327_230706_29_2458'],
 '2021-03-30': ['20210330_231606_45_2233']}


#### Step 4: Submit Orders

Now that we have the scene ids for each collect date, we can create the orders for each date. The output of each order is a single zip file that contains one composited scene and one composited UDM2.

For this step we will just use the python api. See part 1 for a demonstration of how to use the CLI.

##### Step 4.1: Build Order Requests

In [12]:
def build_order(ids, name, aoi_geom):
    # specify the psscene4band surface reflectance product
    # make sure to get the *_udm2 bundle so you get the udm2 product
    # note: capitalization really matters in item_type when using planet client orders api
    item_type = 'PSScene4Band'
    bundle = 'analytic_sr_udm2'

    orders_request = {
        'name': name,
        'products': [{
            'item_ids': ids,
            'item_type': item_type,
            'product_bundle': bundle
        }],
        'tools': get_tools(aoi_geom),
        'delivery': {
            'single_archive': True,
            'archive_filename':'{{name}}_{{order_id}}.zip',
            'archive_type':'zip'
        },
            'notifications': {
                       'email': False
        },
    }
    return orders_request

def get_tools(aoi_geom):
    # clip to AOI
    clip_tool = {'clip': {'aoi': aoi_geom}}
    '''
    # convert to NDVI
    bandmath_tool = {'bandmath': {
        "pixel_type": "32R",
        "b1": "(b1)"
    }}
    
    # convert to NDVI
    bandmath_tool = {'bandmath': {
        "pixel_type": "32R",
        "b1": "(b4 - b3) / (b4+b3)"
    }}
    '''
    

    # composite into one image
    composite_tool = {  
       "composite":{}
    }

    #tools = [clip_tool, bandmath_tool, composite_tool]
    tools = [clip_tool,composite_tool]
    return tools

# uncomment to see what an order request would look like
# pprint(build_order(['id'], 'demo', test_aoi_geom), indent=4)

In [13]:
def get_orders_requests(ids_by_date, aoi_geom):
    order_requests = [build_order(ids, date, aoi_geom)
                      for date, ids in ids_by_date.items()]
    return order_requests
    
order_requests = get_orders_requests(ids_by_date, test_aoi_geom)
print(len(order_requests))
pprint(order_requests[0])

9
{'delivery': {'archive_filename': '{{name}}_{{order_id}}.zip',
              'archive_type': 'zip',
              'single_archive': True},
 'name': '2021-03-13',
 'notifications': {'email': False},
 'products': [{'item_ids': ['20210313_220118_104e'],
               'item_type': 'PSScene4Band',
               'product_bundle': 'analytic_sr_udm2'}],
 'tools': [{'clip': {'aoi': {'coordinates': [[[152.88312435150146,
                                               -27.382590190112836],
                                              [152.87668704986572,
                                               -27.3816375134553],
                                              [152.86853313446045,
                                               -27.386553236946078],
                                              [152.87179470062256,
                                               -27.393107195074954],
                                              [152.873854637146,
                                         

In [14]:
order_requests1 = order_requests[:20]
order_requests2 = order_requests[20:]


##### Step 4.2: Submit Orders

In this section, for the sake of demonstration, we limit our orders to 2. Feel free to increase this limit if you want!

In [16]:
def create_orders(order_requests1, client):
    orders_info = [client.create_order(r).get()
                   for r in order_requests1]
    order_ids = [i['id'] for i in orders_info]
    return order_ids

# testing: lets just create two orders
order_limit = 100
order_ids = create_orders(order_requests[:order_limit], client)
order_ids

['ada13fa5-a804-4d6a-bda5-b0af790c0840',
 '461a28f1-f1fe-4feb-9b2b-a82a6ae54748',
 '24f57c3e-ce12-46bc-adfc-1852e237d2b6',
 '73c57182-562d-4fea-ba61-7b94970e951f',
 'b0d542b1-3b51-4a56-b338-ca9d655f9252',
 'bfd8f1c2-d8e2-4810-81d9-f1fd64583547',
 '31405070-60d9-41ce-b9b5-f72dfb309a03',
 '945b2aa1-f651-4b2c-b3a2-ea52ae94dfaa',
 '384f5bce-045c-41ae-ab5f-916dd8ee1a96']

#### Step 5: Download Orders

##### Step 5.1: Wait Until Orders are Successful

Before we can download the orders, they have to be prepared on the server.

In [17]:
def poll_for_success(order_ids, client, num_loops=50):
    count = 0
    polling = copy(order_ids)
    completed = []
    while(count < num_loops):
        count += 1
        states = []
        for oid in copy(polling):
            order_info = client.get_individual_order(oid).get()
            state = order_info['state']
            states += state
            print('{}:{}'.format(oid, state))
            success_states = ['success', 'partial']
            if state == 'failed':
                raise Exception(response)
            elif state in success_states:
                polling.remove(oid)
                completed += oid
        if not len(polling):
            print('done')
            break
        print('--')
        time.sleep(30)
        
poll_for_success(order_ids, client)

ada13fa5-a804-4d6a-bda5-b0af790c0840:running
461a28f1-f1fe-4feb-9b2b-a82a6ae54748:running
24f57c3e-ce12-46bc-adfc-1852e237d2b6:running
73c57182-562d-4fea-ba61-7b94970e951f:running
b0d542b1-3b51-4a56-b338-ca9d655f9252:running
bfd8f1c2-d8e2-4810-81d9-f1fd64583547:running
31405070-60d9-41ce-b9b5-f72dfb309a03:running
945b2aa1-f651-4b2c-b3a2-ea52ae94dfaa:running
384f5bce-045c-41ae-ab5f-916dd8ee1a96:running
--
ada13fa5-a804-4d6a-bda5-b0af790c0840:running
461a28f1-f1fe-4feb-9b2b-a82a6ae54748:running
24f57c3e-ce12-46bc-adfc-1852e237d2b6:running
73c57182-562d-4fea-ba61-7b94970e951f:running
b0d542b1-3b51-4a56-b338-ca9d655f9252:running
bfd8f1c2-d8e2-4810-81d9-f1fd64583547:running
31405070-60d9-41ce-b9b5-f72dfb309a03:running
945b2aa1-f651-4b2c-b3a2-ea52ae94dfaa:running
384f5bce-045c-41ae-ab5f-916dd8ee1a96:running
--
ada13fa5-a804-4d6a-bda5-b0af790c0840:running
461a28f1-f1fe-4feb-9b2b-a82a6ae54748:running
24f57c3e-ce12-46bc-adfc-1852e237d2b6:running
73c57182-562d-4fea-ba61-7b94970e951f:running
b0d5

##### Step 5.2: Run Download

For this step we will use the planet python orders API because we want to be able to download multiple orders at once, something the CLI does not yet support.

In [18]:
data_dir = "G:\\03_scratchTemp\\scratch\\planet\\samford\\"

# make the download directory if it doesn't exist
#Path(data_dir).mkdir(parents=True, exist_ok=True)

In [19]:
def poll_for_download(dest, endswith, num_loops=50):
    count = 0
    while(count < num_loops):
        count += 1        
        matched_files = (f for f in os.listdir(dest)
                         if os.path.isfile(os.path.join(dest, f))
                         and f.endswith(endswith))

        match = next(matched_files, None)
        if match:
            match = os.path.join(dest, match)
            print('downloaded')
            break
        else:
            print('waiting...')
        time.sleep(10)
    return match

In [21]:
def download_orders(order_ids, client, dest='.', limit=None):
    files = []
    for order_id in order_ids:
        print('downloading {}'.format(order_id))
        filename = download_order(order_id, dest, client, limit=limit)
        if filename:
            files.append(filename)
    return files

def download_order(order_id, dest, client, limit=None):
    '''Download an order by given order ID'''
    # this returns download stats but they aren't accurate or informative
    # so we will look for the downloaded file on our own.
    dl = downloader.create(client, order=True)
    urls = client.get_individual_order(order_id).items_iter(limit=limit)
    dl.download(urls, [], dest)
    
    endswith = '{}.zip'.format(order_id)
    filename = poll_for_download(dest, endswith)
    return filename

downloaded_files = download_orders(order_ids, client, data_dir)

downloading ada13fa5-a804-4d6a-bda5-b0af790c0840
waiting...
downloaded
downloading 461a28f1-f1fe-4feb-9b2b-a82a6ae54748
waiting...
downloaded
downloading 24f57c3e-ce12-46bc-adfc-1852e237d2b6
waiting...
downloaded
downloading 73c57182-562d-4fea-ba61-7b94970e951f
waiting...
downloaded
downloading b0d542b1-3b51-4a56-b338-ca9d655f9252
waiting...
downloaded
downloading bfd8f1c2-d8e2-4810-81d9-f1fd64583547
waiting...
downloaded
downloading 31405070-60d9-41ce-b9b5-f72dfb309a03
waiting...
downloaded
downloading 945b2aa1-f651-4b2c-b3a2-ea52ae94dfaa
waiting...
downloaded
downloading 384f5bce-045c-41ae-ab5f-916dd8ee1a96
waiting...
downloaded


In [23]:
downloaded_files

['G:\\03_scratchTemp\\scratch\\planet\\samford\\2021-03-13_ada13fa5-a804-4d6a-bda5-b0af790c0840.zip',
 'G:\\03_scratchTemp\\scratch\\planet\\samford\\2021-03-07_461a28f1-f1fe-4feb-9b2b-a82a6ae54748.zip',
 'G:\\03_scratchTemp\\scratch\\planet\\samford\\2021-03-25_24f57c3e-ce12-46bc-adfc-1852e237d2b6.zip',
 'G:\\03_scratchTemp\\scratch\\planet\\samford\\2021-03-27_73c57182-562d-4fea-ba61-7b94970e951f.zip',
 'G:\\03_scratchTemp\\scratch\\planet\\samford\\2021-03-30_b0d542b1-3b51-4a56-b338-ca9d655f9252.zip',
 'G:\\03_scratchTemp\\scratch\\planet\\samford\\2021-02-20_bfd8f1c2-d8e2-4810-81d9-f1fd64583547.zip',
 'G:\\03_scratchTemp\\scratch\\planet\\samford\\2021-03-04_31405070-60d9-41ce-b9b5-f72dfb309a03.zip',
 'G:\\03_scratchTemp\\scratch\\planet\\samford\\2021-02-22_945b2aa1-f651-4b2c-b3a2-ea52ae94dfaa.zip',
 'G:\\03_scratchTemp\\scratch\\planet\\samford\\2021-03-24_384f5bce-045c-41ae-ab5f-916dd8ee1a96.zip']

#### Step 6: Unzip and Verify Orders

In this step we will simply unzip the orders and view one of the ordered composite images.

##### 6.1: Unzip Order

In this section, we will unzip each order into a directory named after the downloaded zip file.

In [24]:
def unzip(filename, overwrite=False):
    location = Path(filename)
    zipdir = location.parent / location.stem

    if os.path.isdir(zipdir):
        if overwrite:
            print('{} exists. overwriting.'.format(zipdir))
            shutil.rmtree(zipdir)
        else:
            raise Exception('{} already exists'.format(zipdir))
        
    with ZipFile(location) as myzip:
        myzip.extractall(zipdir)
    return zipdir

zipdirs = [unzip(f, overwrite=True) for f in downloaded_files]
pprint(zipdirs)

[WindowsPath('G:/03_scratchTemp/scratch/planet/samford/2021-03-13_ada13fa5-a804-4d6a-bda5-b0af790c0840'),
 WindowsPath('G:/03_scratchTemp/scratch/planet/samford/2021-03-07_461a28f1-f1fe-4feb-9b2b-a82a6ae54748'),
 WindowsPath('G:/03_scratchTemp/scratch/planet/samford/2021-03-25_24f57c3e-ce12-46bc-adfc-1852e237d2b6'),
 WindowsPath('G:/03_scratchTemp/scratch/planet/samford/2021-03-27_73c57182-562d-4fea-ba61-7b94970e951f'),
 WindowsPath('G:/03_scratchTemp/scratch/planet/samford/2021-03-30_b0d542b1-3b51-4a56-b338-ca9d655f9252'),
 WindowsPath('G:/03_scratchTemp/scratch/planet/samford/2021-02-20_bfd8f1c2-d8e2-4810-81d9-f1fd64583547'),
 WindowsPath('G:/03_scratchTemp/scratch/planet/samford/2021-03-04_31405070-60d9-41ce-b9b5-f72dfb309a03'),
 WindowsPath('G:/03_scratchTemp/scratch/planet/samford/2021-02-22_945b2aa1-f651-4b2c-b3a2-ea52ae94dfaa'),
 WindowsPath('G:/03_scratchTemp/scratch/planet/samford/2021-03-24_384f5bce-045c-41ae-ab5f-916dd8ee1a96')]


##### 6.2: Verify Orders

In this section we will view the orders manually in QGIS. In the next part of this tutorial, we will visualize the NDVI composite image with the UDM. But for now, we just want to make sure we got what we ordered.

In the explorer, navigate to the data folder (should be `*/notebooks/analysis-ready-data/data/use_case_1/`). In any of the subfolders (named something like `2019-04-26_b219cbe1-79a1-49c6-87a0-e1416d7e761d`), go into `files` and find the file named `composite.tif`). Drag that file into QGIS to visualize.